Group Memebers

*   Chirag Gupta - 10623123
*   Chukwuebuka Uwaoma - 10621864
*   Dipanshu



In [161]:
# Import necesarry libraries
import tweepy
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import mysql.connector
from datetime import datetime
import re

In [162]:
# Load dotenv and read secret keys and tokens
load_dotenv()
api_key = os.getenv('api_key')
api_secret_key = os.getenv('api_secret_key')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

In [163]:
# Connect to twitter using tweepy API and stored tokens
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [164]:
# Pulls tweets based on search query and geocode location
tweetlist = []
geocode="53.1424,-7.6921,300km" # "53.3498053,-6.2603097,25km"
filter =  '-filter:retweets AND -filter:replies'
query = 'housing'
tweets = tweepy.Cursor(api.search_tweets, q=query+filter, geocode=geocode, lang='en', tweet_mode="extended", count=20).items(20)

# adds tweets to a dataframe
column_names = ["id", "created_at", "username", "tweet", "retweet_count", "favorite_count", "user_location"]
df = pd.DataFrame(columns=column_names)

for tweet in tweets:
    #create a new row object
    new_row = pd.Series({
        "id": tweet.id, 
        "created_at":tweet.created_at,
        "username": tweet.user.screen_name, 
        "tweet": tweet.full_text, 
        "retweet_count": tweet.retweet_count, 
        "favorite_count": tweet.favorite_count,
        "user_location":tweet.user.location,
    })
    
    # get the index of the last row
    last_row_index = len(df)
    # add the new row to the DataFrame using .iloc[]
    df.loc[last_row_index] = new_row
df.head()
    
# Pull tweet text, id, created_at_date,  username, retweets, likes saved to temporary dataframe - Virgil - DONE

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647635355229724672,2023-04-16 16:17:34+00:00,luke_oriordan,Minister for Housing must explain department u...,0,1,"Glasnevin, Dublin"
1,1647634340082335750,2023-04-16 16:13:32+00:00,UrbanArchCon,6 Collective Housing Interventions that Rethin...,0,0,"South Dublin, Ireland"
2,1647634220729221121,2023-04-16 16:13:04+00:00,Seamus_B_89,A very healthy &amp; very proud looking pair o...,0,0,"Waterford, Ireland"
3,1647630314058166273,2023-04-16 15:57:33+00:00,WaterfordLive,Minister visits Social and Affordable Housing ...,0,0,"Waterford, Ireland"
4,1647625334882271232,2023-04-16 15:37:45+00:00,FancyVegasPro,It would seem like a challenge to vent your fr...,5,38,"Dublin City, Ireland"


In [165]:
# Clean the data
    ## clean text 
    ## date_time in right format eg dd/mm/yyyy - Chirag

# connect to a external DB upload cleaned data
    ## make sure tweets are unique by id - Dipanshu

# pull from database assign to dataframe - DP 
    # tokenize - virgil
    # lematize  - Dipanshu
    # visualise word cloud - Chirag

In [166]:
# Pre-processing the data

# Formatting the date to dd/mm/YYYY format
def formatDate(timestamp):
    # As the timestamp retrieved from the api is already in DatetimeIndex form, simply running string format function 
    # of the datetime library and passing the desired format
    formatted_date = timestamp.strftime("%d/%m/%Y")
    return formatted_date

# Getting only the words and numbers from the tweet that contains links, special symbols, emojis, etc.
def cleanTweet(tweet):
    # Converting the string to lowercase and replacing the ' with blank
    removed_apostrophe = tweet.lower().replace("'","")
    # Using regex pattern to replace the links, <html> tags and &amp; symbol (seen frequently in the tweets) with blank
    removed_links_tags_frequentsymbol = re.sub("(https?://\S+)|(<[\w]+>)|(&amp;)","",removed_apostrophe)
    # This regex pattern returns a list that contains all the words and numbers in the tweet
    cleaned_tweet = re.findall("([a-z0-9]+)", removed_links_tags_frequentsymbol)
    cleaned_tweet = " ".join(cleaned_tweet)
    return cleaned_tweet

# Applying respective functions to both columns in the dataset
df["created_at"] = df["created_at"].apply(lambda x: formatDate(x))
df["tweet"] = df["tweet"].apply(lambda x: cleanTweet(x))
df.head()

,id,created_at,username,tweet,retweet_count,favorite_count,user_location
0,1647635355229724672,16/04/2023,luke_oriordan,minister for housing must explain department u...,0,1,"Glasnevin, Dublin"
1,1647634340082335750,16/04/2023,UrbanArchCon,6 collective housing interventions that rethin...,0,0,"South Dublin, Ireland"
2,1647634220729221121,16/04/2023,Seamus_B_89,a very healthy very proud looking pair of buzz...,0,0,"Waterford, Ireland"
3,1647630314058166273,16/04/2023,WaterfordLive,minister visits social and affordable housing ...,0,0,"Waterford, Ireland"
4,1647625334882271232,16/04/2023,FancyVegasPro,it would seem like a challenge to vent your fr...,5,38,"Dublin City, Ireland"


In [167]:
connection = mysql.connector.connect(host='localhost',
                                         database='ca',
                                         user='root',
                                         password='toor')

In [168]:
mySql_insert_query = 'CREATE TABLE if not exists api (id varchar(255) primary key ,created_at varchar(255),username varchar(255),tweet TEXT(50000),retweet_count varchar(255),favorite_count varchar(255),user_location varchar(255));'

In [169]:
cursor = connection.cursor()
cursor.execute(mySql_insert_query)
connection.commit()

In [170]:
for i in range(df.shape[0]):
    s=""
    for j in df.columns:
        s=s+j+","
    s=s[:-1]

    t=""
    for k in df.columns:
        t=t+"'"+str(df[k].iloc[i])+"',"
    t=t[:-1]
    command="INSERT INTO api ("+s+") VALUES ("+t+");"
    print(command)
    try:
        cursor.execute(command)
    except:
        pass
connection.commit()

INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647635355229724672','16/04/2023','luke_oriordan','minister for housing must explain department underspending eoin broin via newstalkfm','0','1','Glasnevin, Dublin');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647634340082335750','16/04/2023','UrbanArchCon','6 collective housing interventions that rethink urban block design archdaily interiordesign architect commercial fitout contractor build design','0','0','South Dublin, Ireland');
INSERT INTO api (id,created_at,username,tweet,retweet_count,favorite_count,user_location) VALUES ('1647634220729221121','16/04/2023','Seamus_B_89','a very healthy very proud looking pair of buzzards have apparently decided that theyll set up shop about 50 100 yards away from the house im in a housing estate that right up at the back behind me theres a very old farmyard that theyve liked the look of','0'